Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


## Import desired packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import statsmodels.api as sm
import gc

from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

from tqdm import tqdm
import itertools
import gc


In [ ]:
# Load images and labels
# We make sure it loads RGB things
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image = Image.open(os.path.join(folder_path, filename)).convert('RGB').resize((224, 224))
            images.append(np.array(image))
            labels.append(label)
    return images, labels

folder_0 = "/content/drive/MyDrive/data/non_hs"
folder_1 = "/content/drive/MyDrive/data/hs"

# Mark non_head_and_shoulder as 0 and head_and_shoulder as 1
images_0, labels_0 = load_images(folder_0, 0)
images_1, labels_1 = load_images(folder_1, 1)

images = np.array(images_0 + images_1)
labels = np.array(labels_0 + labels_1)

# Split data into train and test sets, we also stratify them 
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rescale=1./255
)

datagen.fit(X_train)

### Alexnet

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_alexnet(input_shape=(224, 224, 3), num_classes=1):
    model = tf.keras.Sequential([
        Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape),
        MaxPooling2D((3, 3), strides=(2, 2)),
        Conv2D(256, (5, 5), activation='relu', padding='same'),
        MaxPooling2D((3, 3), strides=(2, 2)),
        Conv2D(384, (3, 3), activation='relu', padding='same'),
        Conv2D(384, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((3, 3), strides=(2, 2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid' if num_classes == 1 else 'softmax')
    ])
    return model

# Create and compile AlexNet model
alexnet = create_alexnet()
alexnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train AlexNet model
alexnet.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test/255, y_test), epochs=10)
test_loss, test_accuracy = alexnet.evaluate(X_test / 255, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/10
33/33 [==============================] - 216s 6s/step - loss: 0.7558 - accuracy: 0.8956 - val_loss: 0.3499 - val_accuracy: 0.9091
Epoch 2/10
33/33 [==============================] - 213s 6s/step - loss: 0.3266 - accuracy: 0.9089 - val_loss: 0.3048 - val_accuracy: 0.9091
Epoch 3/10
33/33 [==============================] - 212s 6s/step - loss: 0.3080 - accuracy: 0.9089 - val_loss: 0.3075 - val_accuracy: 0.9091
Epoch 4/10
33/33 [==============================] - 215s 7s/step - loss: 0.3082 - accuracy: 0.9089 - val_loss: 0.3088 - val_accuracy: 0.9091
Epoch 5/10
33/33 [==============================] - 211s 6s/step - loss: 0.3114 - accuracy: 0.9089 - val_loss: 0.3273 - val_accuracy: 0.9091
Epoch 6/10
33/33 [==============================] - 210s 6s/step - loss: 0.3153 - accuracy: 0.9089 - val_loss: 0.3066 - val_accuracy: 0.9091
Epoch 7/10
33/33 [==============================] - 209s 6s/step - loss: 0.3075 - accuracy: 0.9089 - val_loss: 0.3053 - val_accuracy: 0.9091
Epoch 8/10
33